## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
data = pd.read_csv('data.csv')
m = Mystem()

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

В датафрейме 21525 строк различного типа данных. 
Столбец days_employed имеет некорректные данные для общего трудового стажа в днях. 
В столбцах days_employed и total_income иимеются пропуски.

    От кредитного отдела банка была получена статистика платежеспособности клиентов. Задача выявить зависимости от семейного положения, наличия детей, дохода и целей кредита на факт погашения кредита в срок. Данные получены в виде файла .csv.
    В файле 21525 записей

### Шаг 2. Предобработка данных

Для ответа на поставленные вопросы проверяем данные в колонке о детях

In [4]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Имеются аномальные данные о количестве детей: -1, 20(в большом отрыве)

In [5]:
print('Артефакты в данных о количестве детей составлют {:.1%}'.format((data.loc[data['children'] == 20]['children'].count()+data.loc[data['children'] == -1]['children'].count()) / data['children'].count())) 

Артефакты в данных о количестве детей составлют 0.6%


заполняем артефакты нулевым значением

In [6]:
data['children'].replace(-1 , 0, inplace=True)
data['children'].replace(20 , 0, inplace=True)
data['children'].value_counts()

0    14272
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

Информация об образовании

In [7]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Информация о семейном положении

In [8]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

### Обработка пропусков

Производим проверку функцией isna() с суммированием пропусков

In [9]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [10]:
data_none = data.loc[data['total_income'].isna()]
data_none.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [11]:
data_none[data_none[index] == 72]['education']

NameError: name 'index' is not defined

Имеются 2174 отсутствующих значения в двух столбцах. Посмотрим на данные трудового стажа в 'days_employed'

In [ ]:
data['days_employed'].head(15)

Колонка days_employed имеет невнятные данные с пропусками. Для решения поставленных вопросов эта колонка не понадобится. Удаляем.

In [ ]:
field = 'days_employed'
try:
    data.drop([field], axis='columns', inplace=True)
    print('столбец', field, 'удален')
except:
    print('столбец', field, 'удален ранее')
data.columns

Колонка total_income:

In [ ]:
pass_data = data['total_income'].isna().sum() / data['total_income'].count()
print('Пропусков в данных о доходах:', data['total_income'].isna().sum())
print('Пропуски составляют:{:.0%}'.format(pass_data))
print('Максимальный доход:', data['total_income'].max())
print('Минимальный доход :', data['total_income'].min())
print('Средний доход :', data['total_income'].mean())
median_income = data['total_income'].median()
print('Медиана ежемесячных доходов:', median_income)

Остались пропуски в солбце 'total_income'. Заменяем пропуски медианным значением. Среднее чувствительно к экстремальным значениям. 

In [ ]:
data['total_income'].fillna(median_income, inplace=True)
#data.head(15)

Проверяем на пропуски повторно

In [ ]:
print('Пропусков данных в ежемесячных доходах:', data['total_income'].isna().sum())

### Вывод

Обнаружены пропущеные значения в двух столбцах: общий стаж и общий доход, в одних и тех-же строках. Вероятнее всего это ошибка выгрузки. В колонке трудового стажа данные не соответствуют реальным данным: Много отрицательных значений или значений соответствующим 500 лет и более. Столбец был удален полностью. В поле ежемесячного дохода отсутствующие значения были заменены медианой изза имеющихся экстремальных значений в верхнем диапазоне доходов.

### Замена типа данных

Для лучшего восприятия заменим вещественный тип столбца 'total_income' на числовой

In [ ]:
data['total_income'] = data['total_income'].astype('int')
#data.head()

In [ ]:
data.info()

### Вывод

Для замены типа данных использовался метод astype().  

### Обработка дубликатов

Посмотрим на дубликаты в таблице целиком методом duplicated(). просуммируем результат

In [ ]:
print('Обнаружено дубликатов строк:', data.duplicated().sum())

Удаляем дубликаты методом drop_duplicates() с параметром inplace=True для фактического удаления повторяющихся строк

In [ ]:
data.drop_duplicates(inplace=True)

Проверяем дубликаты повторно

In [ ]:
print('Обнаружено дубликатов строк:', data.duplicated().sum())

Проверяем категориальные данные на дубликаты, приводим в порядок столбец 'education'

In [ ]:
print(data['education'].value_counts())

Значения находятся в различном регистре. Приводим к нижнему регистру методом  str.lower(). Проверяем результат.

In [ ]:
data['education'] = data['education'].str.lower()
print(data['education'].value_counts())

### Вывод

Обнаружены дубликаты строк полностью, которые могли появиться по техническим причинам. Также Обработаны дубликаты с одним значением, с разным регистром, природа появления которых, вероятно на этапе сбора данных.

### Лемматизация

Для ответа на вопросы имеется необходимость лемматизировать колонку 'purpose' о целях получения кредита.

In [ ]:
print(data['purpose'].value_counts())

Изучив данные о целях получения кредита, значения можно разделить на 4 категории: 'свадьба' 'недвижимость' 'автомобиль' 'образование'. Добавим столбец лемм 'purpose_lemm' с категориями в таблицу с помощью функции, которая разберет списки лемм, объединенных методом join() от каждой строки на категории 'свадьба', 'недвижимость', 'автомобиль', 'образование'. Леммы "жильё" также отнесутся к 'недвижимость'.

In [ ]:
def lemm_purpose(row):
    lemmas = ' '.join(m.lemmatize(row))
    if 'свадьба' in lemmas:
        return 'свадьба'
    elif ('жилье' or 'недвижимость') in lemmas:        
        return 'недвижимость'    
    elif 'автомобиль' in lemmas:    
        return 'автомобиль'   
    elif 'образование' in lemmas:    
        return 'образование'
        
data['purpose_lemm'] = data['purpose'].apply(lemm_purpose)


Результат лемматизации в колонке 'purpose_lemm':

In [ ]:
print(data['purpose_lemm'].value_counts())

In [ ]:
data.head()

### Вывод

Приведение строк к лемме проведено с помощью библиотеки pymystem3. mystem  это морфологический анализатор русского языка с поддержкой снятия морфологической неоднозначности. Т.е. все слова в строках приводятся в именительный падеж, единственное число, мужской род, а глаголы, причастия и деепричастия в инфинитиве несовершенного рода. Pymystem3 по умолчанию выдает список лемматизированных слов, который с помощью функции lemm_purpose(row) категоризируем и записывем в новое поле.

### Категоризация данных

#### Долг и дети

Создаем таблицу 'children_debt' с данными о детях и задолженностях.

In [ ]:
children_debt = data.pivot_table('debt', index='children', aggfunc='sum') #Сводная таблица с суммой невозвратов 
children_debt['count_take'] = data['children'].value_counts() #Количество займов Всего
children_debt['non_return'] = children_debt['debt'] / children_debt['count_take']#Доля невозвратов
#children_debt

#### Долг и семейное положение

Создаем сводную таблицу: Индекс - 'family_status', колонки - данные из 'debt' .
вычисляем отношение должников к общему количеству

In [ ]:
family_debt_pivot = data.pivot_table(index=['family_status'], values='family_status_id', columns= 'debt' ,aggfunc='count')
family_debt_pivot['ratio'] = (family_debt_pivot[1] / (family_debt_pivot[0] + family_debt_pivot[1]))
family_debt_pivot.head()

#### Долг и доходы

Создадим категории дохода в таблице data в столбце 'income_level' по данным поля 'total_income': 
менее 90000 низкий доход;
от 90000 до 160000 - средний доход;
от 160000 до 250000 - выше среднего;
более 250000 - высоких доход

In [ ]:
def income_level(row):
    income = row['total_income'] 
    if income >= 250000:
        return 'Высокий доход'
    elif 250000 > income >= 160000:
        return 'Выше среднего'
    elif 160000 > income >= 90000:
        return 'Средний доход'
    elif 90000 > income:
        return 'Низкий доход'
    
data['income_level'] = data.apply(income_level, axis=1)
#data['income_level'].value_counts()
#data.head()

Создаем сводную таблицу: Индекс - 'income_level', колонки - данные из 'debt' . вычисляем отношение должников к общему количеству

In [ ]:
income_pivot = data.pivot_table(index=['income_level'], values='total_income', columns= 'debt' ,aggfunc='count')
income_pivot['ratio'] = (income_pivot[1] / (income_pivot[0] + income_pivot[1]))
#income_pivot

#### Долг и цели

Создадим сводную таблицу по мемметизированным целям кредита, добавим долю задолженности в последнем столбце

In [ ]:
purpose_pivot = data.pivot_table(index=['purpose_lemm'], values='total_income', columns= 'debt' ,aggfunc='count')
purpose_pivot['ratio'] = (purpose_pivot[1] / (purpose_pivot[0] + purpose_pivot[1]))
#purpose_pivot

### Вывод

Данные подготовлены для ответов на поставленные вопросы. По каждой категории составлены таблицы для удобного отображения.
Объединенины избранные данные в произвольныx группах по заданному критерию.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
childfree = children_debt.loc[0, 'non_return']
print('Невозврат кредитов в срок среди бездетных {:.2%}'.format(childfree))
inchild = children_debt.loc[1:, 'debt'].sum() / children_debt.loc[1:, 'count_take'].sum()
print('Невозврат кредитов в срок среди родителей {:.2%}'.format(inchild))

### Вывод

Невозврат кредитов в срок среди заемщикоов, имеющих детей выше.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
family_debt_pivot.sort_values(by = 'ratio', ascending = True)

### Вывод

Имеется зависимость невозврата. Те, кто был в браке более ответственно относятся к возврату кредита

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
income_pivot.sort_values(by = 'ratio', ascending = True)

### Вывод

Имеется зависимость. Заемщики с высоким и низким доходом отдают кредиты лучше

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
purpose_pivot.sort_values(by = 'ratio', ascending = True)

### Вывод

Наиболее надежные кредиты - кредиты на недвижимость

### Шаг 4. Общий вывод